In [2]:
'''
implement keeloq decryption but with printing out a specific round
use numpy to perform correlation with power trace
'''
# base on https://eprint.iacr.org/2008/058.pdf 
# Physical Cryptanalysis of KeeLoq Code Hopping Applications

import numpy as np
import numpy.linalg as la
import datetime
import struct
import csv
from time import time
import matplotlib.pyplot as plt
import random
import math
from operator import itemgetter
import pandas as pd 


0


In [506]:
KeeLoq_NLF=0x3A5C742E
 
def bit(x,n):
    ''' Get the n'th bit from x'''
    y=(((x)>>(n))&1)
    return y
    
def g5(x,a,b,c,d,e):
    ''' Non linear mixing of bits'''
    y=(bit(x,a)+bit(x,b)*2+bit(x,c)*4+bit(x,d)*8+bit(x,e)*16)
    return y


def read_cipher():
        ciphertext=[]
        ciphertextfile=open(r"./25C3_HCS301/ciphertexts.txt","r")
        for line in ciphertextfile:
            ciphertext.append(int(line,16))
        #print(ciphertext[888])
        #line 889 index starts at 0
        ciphertextfile.close()
        return ciphertext
        
def read_powertrace():
    Peak=[]
    for i in range(997):
        Peak.append(i)
        Peak[i]=[]
        with open(f"./25C3_HCS301/Peak{i+1}.dat",mode='rb') as file:
            for j in range(1650):
                byte = file.read(8)
                Peak[i].append(struct.unpack('d',byte)[0])
        file.close()
    return Peak

def swap_device_key_decrypt_key(device_key): 
    #take 64 and then make sure the first bit is the last bit used in decrypting 
    return(device_key[-17::-1] + device_key[:-17:-1])
        
def key_register(key):
    #take a base 10 digit convert it into the actually key in the right format (flipped and left rotated 16 bits)
    key = bin(key)[2:] #get rid of 0b
    key = key.zfill(64)#64 chars
    key = key[-17::-1] + key[:-17:-1]
    key = int(key,2)   #convert to binary
    return key
  

def bin_rep(x, width = 16):
    ''' binary representation'''
    return(bin(x)[2:].zfill(width))


def keeloq_encrypt(plain,key):
    temp = plain
    for r in range(528) :
        temp = (temp>>1) ^ ( (bit(temp,0)^bit(temp,16)^bit(key,r&63)^bit(KeeLoq_NLF,g5(temp,1,9,20,26,31)))<<31)
    return(temp&0xFFFFFFFF)

def decrypt_round(cipher,device_key,decrypt_round):
    '''one round of a keeloq decryption'''
    #The key starts at bit 15 and then gets left shifted (next bit used is 14)
    return ((cipher<<1) ^bit(cipher,31) ^bit(cipher,15) ^bit(device_key,(15-decrypt_round)&63) ^bit(KeeLoq_NLF,g5(cipher,0,8,19,25,30))) & 0xFFFFFFFF
        
def partial_decrypt (cipher,device_key,stop):
    '''do a partial decryption'''
    for r in range(stop):
        cipher = decrypt_round(cipher,device_key,r)
    
    return cipher&0xFFFFFFFF #y0

def resume_decrypt (cipher,device_key,start,stop):
    '''start in the middle of a decrypt and stop whenever'''
    for r in range(start,stop):
        cipher = decrypt_round(cipher,device_key,r)
    
    return cipher&0xFFFFFFFF #y0

def keeloq_decrypt(cipher,device_key):
    '''A full decrypt is 528 rounds'''
    return partial_decrypt(cipher,device_key,528)

def partial_decrypt_power(cipher,device_key,stop):
    ''' Get the power usage after a partial decrypt'''
    for r in range(stop):
        if r == stop - 1 : y1 = cipher&0xFFFFFFFF
        cipher = decrypt_round(cipher,device_key,r)
        if r == stop - 1 : y0 = cipher&0xFFFFFFFF
        
    #the power output is the hamming distance between the last two rounds    
    power = bin(y1^y0).count('1')

    return power
        
def partial_decrypt_power_array(cipher,device_key,stop):
    ''' Get the history of power usage when decrypting '''
    power = []
    prev = cipher
    try:
        for r in range(stop):
            cipher = decrypt_round(cipher,device_key,r)
            power.append((bin(prev^cipher)).count('1')) #hamming distance between this and previous round
            prev = cipher

        return power[::-1]        
    except Exception as e:
        print(cipher,type(cipher),device_key,type(device_key),e)
        
def partial_decrypt_power_array_with_noise(cipher,device_key,stop):
    p = partial_decrypt_power_array(cipher,device_key,stop)
    power = [random.gauss(phyp,0.01*phyp) for phyp in p ]
    #random guassian noise is added to simulate reality, although we dont know what kind of noise exists in reality, most likely variance it is proportional to peak power 
    return power



In [468]:
#Grey code
#0000001000011000 1010001110010010 1100110100111101 0101110110111111

test_device_key = b'0000001000011000101000111001001011001101001111010101110110111111'
test_device_key = int(test_device_key,2)

dummy_ciphertexts = [keeloq_encrypt(i,test_device_key) for i in range(1000)] 
ciphertexts_d = {i:{} for i in dummy_ciphertexts}

for c,v in ciphertexts_d.items():
    v['powertrace_sim'] = partial_decrypt_power_array_with_noise(c,test_device_key,528) 

#print(ciphertexts_d[ciphertexts[9]])

In [500]:
'''
Input: m : length of key guess, n: number of surviving key guesses, k: known previous key bits
Output: SurvivingKeys
    1: KeyHyp = {0, 1}^m
    2: for all KeyHypi; 0 ≤ i < 2^m do
        3: Perform CPA on round (528 − m) using PHyp and k
    4: end for
    5: SurvivingKeys = n most probable partial keys of KeyHyp
'''

def algorithm1(key_length_guess, n, ciphertexts , power_traces , prev_keys = [0] , current_bits_guessed = 0 ):
    '''
    guess m bits of the keys (try 2**m) keys, take the best n
    best n is based on the higest correlation between the actual power and the power hypothesis based on the guessed keys 
    '''
    
    #put into normal form
    temp_1 = [ key_register(x) for x in  prev_keys]  
    #add more MSB for guessing 
    temp_2 = [  ( (y <<current_bits_guessed) ^ x) for x in temp_1 for y in range(2**key_length_guess)]
    #put back into key register form for decrypting
    keyhyp = [ key_register(x) for x in  temp_2 ]
    #all combinations of test keys and ciphertexts
    keyhyp_d = { (key, ciphertext) : {} for key in keyhyp for ciphertext in ciphertexts}
    
    key_depth = current_bits_guessed + key_length_guess
    
    for key,v in keyhyp_d.items():
        device_keyhyp = key[0]
        ciphertext = key[1]
        power_trace =  power_traces[ciphertext]['powertrace_sim']
        
        v['test_key'] = device_keyhyp  
        v['bin_repr'] = bin(device_keyhyp)[2:].zfill(64)  
        #v['dummy_power'] = partial_decrypt_power_array(ciphertext, device_keyhyp ,key_length_guess)
        v['dummy_power'] = partial_decrypt_power_array(ciphertext, device_keyhyp ,key_depth)[0]
        v['power_trace'] = power_trace[-key_depth]
        #v['dummy_corr'] = np.corrcoef( v['dummy_power'] , power_trace[-key_length_guess : ] )[0,1] 
        #if math.isnan(v['dummy_corr']) : v['dummy_corr'] = 0 

    df = pd.DataFrame([{'test_key':k[0], 'ciphertext':k[1], 'bin_repr': v['bin_repr'], 'dummy_power': v['dummy_power'], 'power_trace': v['power_trace']} for k,v in keyhyp_d.items()])
    #display(t)

    correlation_by_group = df.groupby(['test_key', 'bin_repr']).apply(lambda g: g['dummy_power'].corr(g['power_trace'])).reset_index()
    sorted_corr = correlation_by_group.sort_values(by=0,ascending=False)
            
    return({"likely_keys" : list(sorted_corr['test_key'][0:n]), "current_bits_guessed" : key_depth }) #Get the four most likely( highest max correlation) key guesses

start_time =  datetime.datetime.now()
#t1 = algorithm1(8, 10, dummy_ciphertexts[0:500]  , ciphertexts_d, [151050434977488319          ] , 56   )
#print(len(t1))
#ciphertexts time
#final 8 bits
#100 6 
#200 15
#300 25

#t1 = algorithm1(8, 10, dummy_ciphertexts[0:500] , ciphertexts_d, [0] , 0   )
#t2 = algorithm1(8, 10, dummy_ciphertexts[0:500] , ciphertexts_d, t1["likely_keys"] , t1["current_bits_guessed"]   )
#t3 = algorithm1(4, 10, dummy_ciphertexts[0:500] , ciphertexts_d, t2["likely_keys"][0:5] , t2["current_bits_guessed"]   )
t4 = algorithm1(4, 10, dummy_ciphertexts[0:500] , ciphertexts_d, t3["likely_keys"][0:5] , t3["current_bits_guessed"]   )
print(datetime.datetime.now() - start_time)


0:00:05.781832


In [504]:
start_time =  datetime.datetime.now()
bits = 4 

results = {"likely_keys" : [0], "current_bits_guessed" : 0 }
    
for i in range(64//bits):
    inner_start_time =  datetime.datetime.now()
    results = algorithm1(4, 5, dummy_ciphertexts[0:500] , ciphertexts_d, results["likely_keys"] , results["current_bits_guessed"]   )
    print(datetime.datetime.now() - inner_start_time)
    print([bin(x)[2:].zfill(64)   for x in results])
    
print(datetime.datetime.now() - start_time)
print(results)
    


0:00:00.359293
0:00:02.411955
0:00:02.782451
0:00:03.549847
0:00:04.466597
0:00:06.221057
0:00:06.405609
0:00:07.035303
0:00:07.332248
0:00:08.723260
0:00:09.283909
0:00:10.263033
0:00:11.984063
0:00:12.994593
0:00:12.028303
0:00:12.922697
0:01:58.773835
{'likely_keys': [151050438420815295, 151050438420749759, 151050438420880831, 151050438420946367, 151050438419766719], 'current_bits_guessed': 64}


In [509]:
print(keeloq_decrypt(dummy_ciphertexts[2],test_device_key))

2


In [481]:
def algorithm1_pd(key_length_guess, n, ciphertexts , power_traces , prev_keys = [0] , current_bits_guessed = 0 ):
    #use pandas 
    
    #put into normal form
    temp_1 = [ key_register(x) for x in  prev_keys]  
    #add more MSB for guessing 
    temp_2 = [  ( (y <<current_bits_guessed) ^ x) for x in temp_1 for y in range(2**key_length_guess)]
    #put back into key register form for decrypting
    keyhyp = [ key_register(x) for x in  temp_2 ]
    keyhyp_d = { (key, ciphertext) : {} for key in keyhyp for ciphertext in ciphertexts}

    key_depth = current_bits_guessed + key_length_guess

    key_tests = pd.DataFrame([{ 'test_key':key, 'ciphertext':ciphertext}  for key in keyhyp for ciphertext in ciphertexts], dtype = 'uint64')
    
    #put into binary representaion (string) for easier inspection
    key_tests['bin_repr'] = key_tests['test_key'].apply( lambda x : bin(x)[2:].zfill(64)  )
    
    # get the power trace
    key_tests['power_trace'] = key_tests['ciphertext'].map(lambda ct: power_traces[ct]['powertrace_sim'][-key_depth] if ct in power_traces else None)
    #get the hypothetical power
    key_tests['dummy_power'] =  key_tests.apply(
        lambda row:   partial_decrypt_power_array(int(row['ciphertext']), int(row['test_key']) ,key_depth)[0] , axis=1)    

    return(key_tests) #Get the four most likely( highest max correlation) key guesses

start_time =  datetime.datetime.now()

df = algorithm1_pd(8, 10, dummy_ciphertexts[0:500]  , ciphertexts_d, [0] , 56   )
print(datetime.datetime.now() - start_time)
#display(df)
print(len(df))

0:00:50.656387
128000


In [483]:

correlation_by_group = df.groupby(['test_key', 'bin_repr']).apply(lambda g: g['dummy_power'].corr(g['power_trace'])).reset_index()
#correlation_by_group['keyr'] = correlation_by_group['key'].apply(key_register) 
sorted_corr = correlation_by_group.sort_values(by=0,ascending=False)
print(sorted_corr[0:4])

#for k,v in t1.items():
#    print(v['bin_repr'][48:], v['dummy_corr'])

     test_key                                           bin_repr         0
38    2490368  0000000000000000000000000000000000000000001001...  0.016248
150   9830400  0000000000000000000000000000000000000000100101...  0.016162
36    2359296  0000000000000000000000000000000000000000001001...  0.014847
166  10878976  0000000000000000000000000000000000000000101001...  0.014113
